In [24]:
%matplotlib inline 
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [25]:
from sklearn.datasets import load_iris
iris = load_iris()

X = iris.data  # we only take the first two features.
y = iris.target
# plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1, edgecolor='k')
# plt.xlabel('Sepal length')
# plt.ylabel('Sepal width')

overall_mean = np.mean(X, axis=0)
print('overall mean is:\n ', overall_mean)

class_mean = []
for i in range(3):
    left = i*50
    right  = (i+1)*50
    class_mean.append(np.mean(X[left:right,:], axis=0))
print('class mean is:\n ', class_mean)

S_w = []

overall mean is:  [ 5.84333333  3.054       3.75866667  1.19866667]
class mean is:  [array([ 5.006,  3.418,  1.464,  0.244]), array([ 5.936,  2.77 ,  4.26 ,  1.326]), array([ 6.588,  2.974,  5.552,  2.026])]


In [19]:
overall_mean[0:4]

array([ 5.84333333,  3.054     ,  3.75866667,  1.19866667])

In [22]:
class_mean[1]

array([ 5.936,  2.77 ,  4.26 ,  1.326])

In [27]:
np.shape(overall_mean)

(4,)

In [28]:
np.shape(class_mean)

(3, 4)